# 0. Extract from MXL

In [84]:
import zipfile
import glob

filename = "../../XMLs/jinglebells.xml"
with zipfile.ZipFile(filename, 'r') as zipfile:
    zipfile.extractall('jinglebells.musicxml')

BadZipFile: File is not a zip file

# 1. Read XML

In [4]:
import xml.etree.cElementTree as ET

filename = "../../XMLs/至少還有你_主旋律.musicxml"
tree = ET.parse(filename)

In [5]:
root = tree.getroot()
root.tag, root.attrib

('score-partwise', {'version': '3.1'})

In [6]:
all_notes = root.findall('.//note')

In [7]:
lyric_notes = []
for note in all_notes:
    n = {}
    if note.find('lyric') is not None:
        n['type'] = note.find('type').text
        n['lyric'] = note.find('./lyric/text').text
        n['pitch'] = note.find('./pitch/step').text + note.find('./pitch/octave').text
        if note.find('./pitch/alter') is not None:
            if note.find('./pitch/alter').text == '-1':
                p = n['pitch'][0] + 'b' + n['pitch'][1]
                n['pitch'] = p
            else:
                n['pitch'] = n['pitch'][0] + '#' + n['pitch'][1]
                
        lyric_notes.append(n)

In [8]:
len(lyric_notes)

159

In [9]:
lyric_notes

[{'type': '16th', 'lyric': '我', 'pitch': 'G4'},
 {'type': '16th', 'lyric': '怕', 'pitch': 'G4'},
 {'type': 'eighth', 'lyric': '來', 'pitch': 'F#4'},
 {'type': '16th', 'lyric': '不', 'pitch': 'A4'},
 {'type': '16th', 'lyric': '幾', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '我', 'pitch': 'A4'},
 {'type': '16th', 'lyric': '要', 'pitch': 'A4'},
 {'type': 'eighth', 'lyric': '抱', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '著', 'pitch': 'A4'},
 {'type': '16th', 'lyric': '你', 'pitch': 'B4'},
 {'type': '16th', 'lyric': '直', 'pitch': 'G4'},
 {'type': '16th', 'lyric': '到', 'pitch': 'G4'},
 {'type': 'eighth', 'lyric': '感', 'pitch': 'F#4'},
 {'type': 'eighth', 'lyric': '覺', 'pitch': 'A4'},
 {'type': 'eighth', 'lyric': '你', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '的', 'pitch': 'E5'},
 {'type': '16th', 'lyric': '皺', 'pitch': 'E5'},
 {'type': 'eighth', 'lyric': '紋', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '有', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '了', 'pitch': 'E5'},
 {'type': 'eighth', 'lyric

# 2. Exploring 

In [10]:
time = root.find('.//time')

In [11]:
time.find(".//beats").text

'4'

In [12]:
time.find(".//beat-type").text

'4'

# 3. Helper classes /functions

## 3.1 Helper classes

In [13]:
from enum import Enum

class NoteTypes(Enum):
    '''Class defining types of notes'''
    whole = 1
    half = 2
    quarter = 4
    eight = 8
    sixteenth = 16
    thirtysecond = 32

In [14]:
class NoteLetters(Enum):
    C = 0
    D = 2
    E = 4
    F = 6
    G = 8
    A = 10
    B = 12

In [80]:
import math

class Note:
    '''Class representing a musical note'''
    notetype = None
    step = ''
    octave = 0
    so = None
    tie = None
    accidental = None
    
    def __init__(self,notetype,step,octave,tie,accidental):
        self.notetype = notetype
        self.step = step
        self.octave = int(octave)
        self.tie = tie
        self.accidental = accidental
        
    def get_so(self):
        if self.accidental:
            so = self.step + self.accidental + str(self.octave)
        else:
            
            so = self.step + str(self.octave)
        return so
    
    def printNote(self):
        if self.step=='':
            print('Note: ' + 'rest' + ' / ' +
              str(self.notetype))
        else:
            print('Note: ' + str(self.get_so()) + ' / ' +
              str(self.notetype) + ' / ' +
              str(self.tie))
            
    def addOctaves(self,addOct):
        self.octave += addOct
        
    def copyNote(self):
        ncopy = Note(self.notetype,self.step,self.octave,self.tie,self.accidental)
        return ncopy
    
    def addSemiTones(self,semiTToAdd):
        
        #get relative position of new note
        newVal = getattr(NoteLetters,self.step).value + semiTToAdd
        #DEBUG
        #print(newVal)
        
        #handle semitones first
        if (semiTToAdd%2!=0):
            
            if self.accidental == '#':
                self.accidental = None
                newVal += 1

            elif self.accidental == 'b':
                self.accidental = None
                newVal -= 1
            
            elif not self.accidental:
                self.accidental = '#'
                newVal -= 1
                
            #natural, assumes sharps in score
            else:
                self.accidental = None
                newVal -= 1

       
        #Change step
        self.step = NoteLetters(newVal% 14).name
        
        #change octave
        self.octave += math.floor(newVal/14)   

        

In [16]:
class Measure:
    '''Class representing a measure'''
    notes = []
    
    def __init__(self,notes):
        self.notes = notes
    
    def get_sos(self):
        sos = []
        for note in self.notes:
            sos.append(note.get_so())
        return sos
    
    def get_notetypes(self):
        notetypes = []
        for note in self.notes:
            notetypes.append(note.notetype)
        return notetypes 
    
    def get_ties(self):
        ties = []
        for tie in self.notes:
            ties.append(note.tie)
        return ties 
    
    def printMeasure(self):
        for note in self.notes:
            note.printNote()
            
    def get_notes(self):
        return notes

## 3.2 Helper functions

In [17]:
def get_ts_xml(root):
    '''Gets time signature for a musicxml file.
    Input: Root of XML Tree
    Output:top and bottom numbers of the Time signature'''
    time = root.find('.//time')
    top = int(time.find(".//beats").text)
    bottom = int(time.find(".//beat-type").text)
    return top,bottom

In [18]:
get_ts_xml(root)

(4, 4)

In [19]:
def get_bpm_xml(root):
    '''Gets beats per minute as well as type of beats for a musicxml file.
    Input: Root of XML Tree
    Output: beats per minute, noteType'''
    metronome = root.find('.//metronome')
    beats_per_min = int(metronome.find('.//per-minute').text)
    beat_type = metronome.find('.//beat-unit').text
    #beat_type = getattr(NoteTypes, str(beat_type))
    return beats_per_min, beat_type

In [20]:
get_bpm_xml(root)

(62, 'quarter')

In [21]:
def decode_xml_note(xml_note):
    '''Gets note type, step, octave and tie type'''
    try:
        notetype = xml_note.find('.//type').text
    except:
        notetype = None
    try:
        step = xml_note.find('.//step').text
    except:
        step = ''
    try:
        octave = xml_note.find('.//octave').text
    except:
        octave = 0
    try:
        tie = xml_note.find('.//tie').get('type')
    except:
        tie = None
    try:
        accidental = xml_note.find('.//accidental').text
        if accidental == 'natural':
            accidental = 'n'
        if accidental == 'flat':
            accidental = 'b'
        if accidental == 'sharp':
            accidental = '#'
            
    except:
        accidental = None
        
    note = Note(notetype,step,octave,tie,accidental)
    #DEBUG
    #note.printNote()
    
    return note

In [22]:
measures = root.findall('.//measure')
xml_measure_notes = measures[4].findall('.//note')
xml_note = xml_measure_notes[0]
decode_xml_note(xml_note)

In [23]:
def get_meas_notes_xml(xml_measure_notes):
    ''''return all notes in an xml measure'''
    notes = []
    for xml_note in xml_measure_notes:
        note = decode_xml_note(xml_note)
        if note.notetype:
            notes.append(note)
        #DEBUG
        #note.printNote()
    return notes

In [24]:
def compare_notes(n1,n2):
    '''returns the difference in semitones between 2 notes'''
    val1 = getattr(NoteLetters,n1.step).value
    val2 = getattr(NoteLetters,n2.step).value
    difference = val2 - val1
    #add octaves
    difference += (n2.octave - n1.octave) * 14
    #add accidentals
    if (n1.accidental):
        if n1.accidental == '#':
            difference-=1
        elif n1.accidental == 'b':
            difference+=1
        #naturals,assumes the sheet music has flat alterations only
        else:
            difference+=1
    if (n2.accidental):
        if n1.accidental == '#':
            difference+=1
        elif n1.accidental == 'b':
            difference-=1
        #naturals,assumes the sheet music has flat alterations only
        else:
            difference-=1
    return difference

In [65]:
def get_inverted_measure(measure):
    
    #get first note
    inv_notes = []
    prevNoteNew = measure.notes[0].copyNote()
    if prevNoteNew.step!= '':
        prevNoteNew.addSemiTones(-14)
    #DEBUG
    #prevNoteNew.printNote()
    inv_notes.append(prevNoteNew)
    prevNoteOg = measure.notes[0].copyNote()
    
    for nextNoteOg in measure.notes[1:]:
        #handle rests
        if nextNoteOg.step == '':
            nextNoteNew = nextNoteOg.copyNote()
        else:
             #if previous measure is a rest
            if prevNoteOg.step!='':
                nextNoteNew = Note(nextNoteOg.notetype,prevNoteNew.step,prevNoteNew.octave,nextNoteOg.tie,nextNoteOg.accidental)
                diff = compare_notes(prevNoteOg,nextNoteOg)
            else:
                nextNoteNew = nextNoteOg.copyNote()
                diff = 14
            nextNoteNew.addSemiTones(-diff)
        inv_notes.append(nextNoteNew)
        prevNoteOg = nextNoteOg
        prevNoteNew = nextNoteNew
            #DEBUG 
            #print(diff)
            #prevNoteNew.printNote()

    inv_meas = Measure(inv_notes)
    return inv_meas
    

In [61]:
get_meas_notes_xml(xml_measure_notes)

## 3.3 Try to use classes

In [27]:
#get a note
xml_measures = root.findall('.//measure')
xml_measure = xml_measures[3].findall('.//note')
xml_note = xml_measure[0]
note = decode_xml_note(xml_note)
note.printNote()

Note: rest / half


In [28]:
note.so

In [29]:
note.notetype

'half'

In [30]:
note.tie

In [31]:
#get a measure
measure = Measure(get_meas_notes_xml(xml_measure))
measure.printMeasure()

Note: rest / half
Note: rest / quarter
Note: rest / eighth
Note: G4 / 16th / None
Note: G4 / 16th / None


In [32]:
measure.get_sos()

['0', '0', '0', 'G4', 'G4']

In [33]:
measure.get_notetypes()

['half', 'quarter', 'eighth', '16th', '16th']

In [34]:
measure.get_ties()

[None, None, None, None, None]

In [35]:
#read all measures
xml_measures = root.findall('.//measure')
measures = []

for xml_measure in xml_measures:
    measure = Measure(get_meas_notes_xml(xml_measure))
    if not measure.get_sos()==['0']:
        measures.append(measure)

In [36]:
#try adding semitones
n1 = measures[1].notes[1]
n1.printNote()
n2 = n1.copyNote()
n2.printNote()

Note: A4 / 16th / None
Note: A4 / 16th / None


## 3.4 Add/remove semitones and compare notes

In [37]:
#new note at the nottom, lower by 2 semitones
n3 = Note('quarter','B','4',None,None)
n3.printNote()
newval = getattr(NoteLetters,n3.step).value+2
newval

Note: B4 / quarter / None


14

In [38]:
NoteLetters(newval%14)

<NoteLetters.C: 0>

In [39]:
NoteLetters(newval%14).name

'C'

In [40]:
#try the function
n4 = Note('quarter','B','4',None,None)
n4.addSemiTones(-2)
n4.printNote()

Note: A4 / quarter / None


In [41]:
#try the function
n4 = Note('quarter','B','4',None,None)
n4.addSemiTones(2)
n4.printNote()

Note: C5 / quarter / None


## 3.5 Compare 2 notes

In [42]:
#1st try
n1 = Note('quarter','B','4',None,None)
n2 = Note('quarter','D','4',None,None)
n3 = n1.copyNote()

In [43]:
diff = compare_notes(n1,n2)
diff

-10

In [44]:
n3.addSemiTones(-diff)
n3.printNote()

Note: G5 / quarter / None


In [45]:
#2nd try
n1 = Note('quarter','D','3',None,None)
n2 = Note('quarter','A','5',None,None)
n3 = n1.copyNote()
n1.printNote()

Note: D3 / quarter / None


In [46]:
diff = compare_notes(n1,n2)
diff

36

In [47]:
n3.addSemiTones(diff)
n3.printNote()

Note: A5 / quarter / None


# 3.6 invert notes

In [66]:
#use function
m1 = measures[4]

In [67]:
m1.printMeasure()

Note: F5 / eighth / None
Note: C5 / 16th / None
Note: C5 / 16th / start
Note: C5 / eighth / stop
Note: B4 / eighth / None
Note: C5 / quarter / None
Note: A4 / 16th / None
Note: A4 / 16th / None


In [68]:
inv_m2 = None
inv_m2 = get_inverted_measure(m1)
inv_m2.printMeasure()

Note: F4 / eighth / None
Note: B4 / 16th / None
Note: B4 / 16th / start
Note: B4 / eighth / stop
Note: C5 / eighth / None
Note: B4 / quarter / None
Note: D5 / 16th / None
Note: D5 / 16th / None


In [69]:
inv_m2 = get_inverted_measure(measures[0])
inv_m2.printMeasure()

Note: rest / half
Note: rest / quarter
Note: rest / eighth
Note: G3 / 16th / None
Note: G3 / 16th / None


In [ ]:
compare_notes(measures[4].notes[0],measures[4].notes[1])

In [81]:
i=0
for measure in measures:
    inv_meas = get_inverted_measure(measure)
    print(i)
    i+=1
    inv_meas.printMeasure()

0
Note: rest / half
Note: rest / quarter
Note: rest / eighth
Note: G3 / 16th / None
Note: G3 / 16th / None
1
Note: F3 / eighth / None
Note: D3 / 16th / None
Note: A2 / 16th / start
Note: A2 / half / stop
Note: A2 / eighth / stop
Note: D3 / 16th / None
Note: D3 / 16th / None
2
Note: D4 / eighth / None
Note: G4 / 16th / None
Note: F4 / 16th / start
Note: F4 / half / stop
Note: F4 / eighth / stop
Note: A4 / 16th / None
Note: A4 / 16th / None
3
Note: F3 / eighth / None
Note: D3 / eighth / None
Note: A2 / eighth / None
Note: G2 / 16th / None
Note: G2 / 16th / start
Note: G2 / 16th / stop
Note: A2 / eighth / None
Note: rest / eighth
Note: D4 / 16th / None
Note: C4 / 16th / None
4
Note: F4 / eighth / None
Note: B4 / 16th / None
Note: B4 / 16th / start
Note: B4 / eighth / stop
Note: C5 / eighth / None
Note: B4 / quarter / None
Note: D5 / 16th / None
Note: D5 / 16th / None
5
Note: B3 / eighth / None
Note: G3 / eighth / None
Note: G3 / eighth / None
Note: B3 / 16th / None
Note: F3 / 16th / start

In [79]:
measures[18].printMeasure()

Note: F5 / eighth / None
Note: E5 / 16th / None
Note: E5 / 16th / start
Note: E5 / eighth / stop
Note: E5 / 16th / None
Note: D5 / 16th / None
Note: E5 / 16th / None
Note: E5 / 16th / None
Note: A5 / eighth / None
Note: A5 / eighth / None
Note: A4 / 16th / None
Note: A4 / 16th / None


# 4. Read XML and get measures to invert chords

In [ ]:
xml_measures = root.findall('.//measure')
measures = []

for xml_measure in xml_measures:
    measure = Measure(get_meas_notes_xml(xml_measure))
    measures.append(measure)
    

In [ ]:
for measure in measures:
    measure.printMeasure()